In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time
import os
from common import *

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
with open('SVHN_train.pickle', 'rb') as f:
    tmp_save = pickle.load(f)
    train_dataset_56 = tmp_save['dataset']
    train_labels = tmp_save['labels']
    
valid_size = 2000
valid_dataset_56 = train_dataset_56[:valid_size]
valid_labels = train_labels[:valid_size]
train_dataset_56 = train_dataset_56[valid_size:]
train_labels = train_labels[valid_size:]

with open('SVHN_test.pickle', 'rb') as f:
    tmp_save = pickle.load(f)
    test_dataset_56 = tmp_save['dataset']
    test_labels = tmp_save['labels']
    
print('Training set', train_dataset_56.shape, train_labels.shape)
print('Validation set', valid_dataset_56.shape, valid_labels.shape)
print('Test set', test_dataset_56.shape, test_labels.shape)   

Training set (31402, 56, 56) (31402,)
Validation set (2000, 56, 56) (2000,)
Test set (13068, 56, 56) (13068,)


In [4]:
num_digits = 3
def trim_max_digits(dataset, labels, max_digits):
    keep = [i for i, label in enumerate(labels) if len(label) <= max_digits]
    return dataset[keep], labels[keep]

train_dataset_56, train_labels = trim_max_digits(train_dataset_56, train_labels, num_digits)
valid_dataset_56, valid_labels = trim_max_digits(valid_dataset_56, valid_labels, num_digits)
test_dataset_56, test_labels = trim_max_digits(test_dataset_56, test_labels, num_digits)
print('Training set', train_dataset_56.shape, train_labels.shape)
print('Validation set', valid_dataset_56.shape, valid_labels.shape)
print('Test set', test_dataset_56.shape, test_labels.shape) 

Training set (30041, 56, 56) (30041,)
Validation set (1917, 56, 56) (1917,)
Test set (12920, 56, 56) (12920,)


In [5]:
def show_image(img, label):
    print("Labels", label)
    print("Dtype", img.dtype)
    print("Shape", img.shape)
    print("Color range", np.min(img), np.max(img))
    if len(img.shape) > 2:
        plt.imshow(np.reshape(img, img.shape[:2]))
    else:
        plt.imshow(img)
    plt.show()
    
def show_images(imgs, labels, num=3):
    for i in range(num):
        num = np.random.randint(imgs.shape[0])
        show_image(imgs[num], labels[num])

In [6]:
num_labels = 11 # Add an extra character so we can deal with spaces
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset_output = dataset.reshape(list(dataset.shape) + [1]).astype(np.float32)
    labels_output = np.array([np.array([(np.arange(num_labels) == l).astype(np.float32) 
                        for l in np.append(row, [num_labels-1] * (num_digits - len(row)), 0)])
              for row in labels])
    return dataset_output, labels_output

train_dataset_56, train_labels = reformat(train_dataset_56, train_labels)
valid_dataset_56, valid_labels = reformat(valid_dataset_56, valid_labels)
test_dataset_56, test_labels = reformat(test_dataset_56, test_labels)

print('Training set', train_dataset_56.shape, train_labels.shape)
print('Validation set', valid_dataset_56.shape, valid_labels.shape)
print('Test set', test_dataset_56.shape, test_labels.shape)

Training set (30041, 56, 56, 1) (30041, 3, 11)
Validation set (1917, 56, 56, 1) (1917, 3, 11)
Test set (12920, 56, 56, 1) (12920, 3, 11)


In [7]:
#Limit test set as it's killing the box
test_dataset_56 = test_dataset_56[:6000]
test_labels = test_labels[:6000]

# Create a pit sized 
train_dataset_28 = train_dataset_56[:, ::2, ::2, :]
valid_dataset_28 = valid_dataset_56[:, ::2, ::2, :]
test_dataset_28 = test_dataset_56[:, ::2, ::2, :]
print('Training set', train_dataset_28.shape, train_labels.shape)
print('Validation set', valid_dataset_28.shape, valid_labels.shape)
print('Test set', test_dataset_28.shape, test_labels.shape)

Training set (30041, 28, 28, 1) (30041, 3, 11)
Validation set (1917, 28, 28, 1) (1917, 3, 11)
Test set (6000, 28, 28, 1) (6000, 3, 11)


In [8]:
dataset_56 = (train_dataset_56, valid_dataset_56, test_dataset_56)
dataset_28 = (train_dataset_28, valid_dataset_28, test_dataset_28)

In [ ]:
train_dataset, valid_dataset, test_dataset = dataset_28

def run(graph_config):
    graph = create_graph(num_digits, num_labels, valid_dataset, test_dataset, **graph_config) 
    run_graph(graph, num_digits, train_dataset, train_labels, valid_labels, test_labels, 
              mins=10, save_model=True)
    return graph

In [ ]:
graph = run({'use_dropout':True,
             'use_max_pool':True,
             'learning_rate':0.25,
             'num_hidden':192,
             'depth1':32})

Initialized
Elapsed time(s):17/600 (0.03%)
Learning rate: 0.249353
Minibatch loss at step 500: 1.374956
Minibatch accuracy: 75.0%
Validation accuracy: 55.2%
Elapsed time(s):38/600 (0.06%)
Learning rate: 0.248708
Minibatch loss at step 1000: 1.343120
Minibatch accuracy: 87.5%
Validation accuracy: 65.8%
Elapsed time(s):58/600 (0.10%)
Learning rate: 0.248066
Minibatch loss at step 1500: 0.829576
Minibatch accuracy: 91.7%
Validation accuracy: 70.5%
Elapsed time(s):79/600 (0.13%)
Learning rate: 0.247424
Minibatch loss at step 2000: 1.161056
Minibatch accuracy: 85.4%
Validation accuracy: 72.0%
Elapsed time(s):99/600 (0.17%)
Learning rate: 0.246785
Minibatch loss at step 2500: 0.690728
Minibatch accuracy: 87.5%
Validation accuracy: 74.8%

In [ ]:
from scipy import ndimage
import scipy.misc as spm
import scipy.ndimage as spn

image_data = ndimage.imread('pic.png')

plt.imshow(image_data)
plt.show()
print(image_data.shape)

# for i in range(4):
#     plt.imshow(image_data[:, :, i])
#     plt.show()

image_data = image_data[:, :, :3]
print(image_data.shape)
image_data = image_data[::2, ::2, :]
image_data = np.reshape(image_data, [1, 28, 28, 3])
print(image_data.shape)

print(np.min(image_data), np.max(image_data))
image_data = (image_data - np.mean(image_data)) / np.std(image_data)
print(np.min(image_data), np.max(image_data))

# for i in range(3):
#     plt.imshow(image_data[0, :, :, i])
#     plt.show()

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
np.set_printoptions(linewidth=200)

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    

r = np.clip(np.array(result), -0.5, 0.5)
print(r)
np.reshape(r, [5, -1])

In [ ]:
with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver(tf.all_variables())
    sess.run(tf.initialize_all_variables())
    
    # Here's where you're restoring the variables w and b.
    # Note that the graph is exactly as it was when the variables were
    # saved in a prior training run.
    
    checkpoint_path = os.path.join('save', 'model.ckpt')
    ckpt = tf.train.get_checkpoint_state('save')
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        
    predict_single_output = graph.get_tensor_by_name('predict_single_output:0')
    tf_predict_single_dataset = graph.get_tensor_by_name('tf_predict_single_dataset:0')
    feed_dict = {tf_predict_single_dataset : image_data}
    result = predict_single_output.eval(feed_dict=feed_dict)
                
    tf_train_dataset = graph.get_tensor_by_name('tf_train_dataset:0')
    tf_train_labels = [graph.get_tensor_by_name('tf_train_labels_%d:0' % i) for i in range(num_digits)]
    train_prediction = [graph.get_tensor_by_name('train_prediction_%d:0' % i) for i in range(num_digits)]
    valid_prediction = [graph.get_tensor_by_name('valid_prediction_%d:0' % i) for i in range(num_digits)]
    test_prediction = [graph.get_tensor_by_name('test_prediction_%d:0' % i) for i in range(num_digits)]
    optimizer = graph.get_tensor_by_name('optimizer:0')
    loss = graph.get_tensor_by_name('loss:0')
    learning_rate = graph.get_tensor_by_name('learning_rate:0')
            
    results = sess.run(test_prediction)
    

In [ ]:
with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver(tf.all_variables())
    sess.run(tf.initialize_all_variables())
    
    # Here's where you're restoring the variables w and b.
    # Note that the graph is exactly as it was when the variables were
    # saved in a prior training run.
    
    checkpoint_path = os.path.join('save', 'model.ckpt')
    ckpt = tf.train.get_checkpoint_state('save')
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        
    predict_single_output = graph.get_tensor_by_name('predict_single_output:0')
    tf_predict_single_dataset = graph.get_tensor_by_name('tf_predict_single_dataset:0')
    feed_dict = {tf_predict_single_dataset : image_data}
    result = predict_single_output.eval(feed_dict=feed_dict)

In [ ]:
m = np.reshape(result, [5, -1])
print(np.apply_along_axis(np.argmax, 1, m))

In [ ]:
r = np.array(results)
r.shape

for i in range(110, 110):
    img = test_dataset[i]
    plt.imshow(np.reshape(img, img.shape[:2]))
    plt.show()
    print(np.apply_along_axis(np.argmax, 1, test_labels[i]))
    n = r[:, i, :]
    print(np.apply_along_axis(np.argmax, 1, n))

In [ ]:
image_data.shape

In [ ]:
img_height, img_width = 28, 28

def graph_play():
    graph = tf.Graph()
    
    with graph.as_default():

         # Create a single input tensor for use in the phone
        tf_predict_single_dataset = tf.placeholder(tf.float32, shape=(1,
                                                           img_height,
                                                           img_width,
                                                           3), name="input")
        
        conv = tf.reduce_mean(tf_predict_single_dataset, 3)
        print(conv.get_shape().as_list())
        # Shape it correctly
        conv = tf.reshape(conv, conv.get_shape().as_list() + [1], name='output')
        # Move the color range
        print(conv.get_shape().as_list())
        conv = tf.add(tf.div(conv, 255), -0.5, name='output')
        print(conv.get_shape().as_list())

    return graph

def run(graph):
    
    session = tf.Session(graph=graph);
    
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        
        tf_input = graph.get_tensor_by_name('input:0')
        
        tf_output = graph.get_tensor_by_name('output:0')
        
        feed_dict = {tf_input : image_data}        
        
        result = tf_output.eval(feed_dict=feed_dict)
        
    return result
        
result = run(graph_play())

In [ ]:
result.shape